In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'videos_CNN_large'

In [3]:
config_path = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME}.yaml")
config = load_config(config_path)

In [4]:
model = CondRealNVP.from_config(config).to(device)

print(f'{model.n_params:,}')

67,691,783


In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/psaegert/Projects/bcnf/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 1/1 [00:01<00:00,  1.65s/it, file=fixed_data_render_2s_15FPS_1.pkl]


Using videos data for training. Shapes:
X shape: torch.Size([1000, 2, 30, 90, 160])
y shape: torch.Size([1000, 19])


In [6]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -29.6699 - Val: -21.6093 (avg: -21.6653, min: -22.0063) | lr: 1.00e-04 - Patience: 104/500 - z: (-0.2623 ± 0.4980) ± (1.2327 ± 0.3919):   3%|▎         | 1561/50000 [1:23:01<42:11:56,  3.14s/it]

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/psaegert/Projects/bcnf/models/bcnf-models/trajectory_FC_small
